## Домашняя работа № 1

Данг Куинь Ньы, БПМИ182

### Задание 2

In [1]:
import numpy as np

[scipy: base class to construct specific distribution classes and instances for continuous random variables](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html)

In [2]:
from scipy.stats import rv_continuous
class custom_gen(rv_continuous):
    def _pdf(self, x, q):
        if not (x > 0): return 0    # check if x > 0
        return (2 * x / q) * np.exp(-1 / q * (x ** 2))

In [3]:
# generate sample
unf_curses = custom_gen(name='unforgivable curses')
x2 = unf_curses.rvs(size=1273, q=5)    # true value of q = 5, sample size = 1273

In [4]:
# log of likelihood function
def loglike(q, x):
    n = len(x)
    return n * np.log(2 / q) + np.sum(np.log(x)) - np.sum(x**2) / q

In [5]:
# optimization of loglike function
def opt_loglike(q, x):    
    return -loglike(q, x)

Найдём оценку $\hat{q}$. Поскольку $f(x) = x^2$ - гладкая функция, оценка $\widehat{q^2} = (\hat{q})^2$.

In [6]:
from scipy import optimize
q_hat = optimize.minimize(opt_loglike, 0.001, x2, method = 'Nelder-Mead')['x'][0]
print('Computed estimate of q and q**2 values: %.4f %.4f' % (q_hat, q_hat ** 2))

Computed estimate of q and q**2 values: 4.9926 24.9256


Для сравнения выведем теоретическую оценку параметра $\hat{q} = \overline{X^2}$:

In [7]:
q_hat_theory = np.mean(x2 ** 2)
print('Calculated estimate of q and q**2 values: %.4f %.4f' % (q_hat_theory, q_hat_theory ** 2))

Calculated estimate of q and q**2 values: 4.9925 24.9254


Функцию правдоподобия (точнее, логарифм от неё) оптимизировали, взяв её со знаком минус (аналогично оптимизации в семинаре 1). Оценка находится уже минимизацией преобразованной функции, дополнительных трудностей не возникло. Как видно, оценка (с начальным приближением 0.001) находится довольно точно - при истинном значении 5 выдаётся оценка 4.9926.

### Задание 4

Симулируем 300 наблюдений ($x_i, y_i$) таким образом, что $x_i \sim \mathcal{N}(0, 1), \, \, \,y_i = 3e^{x_i}u_i, \, \, \, \ln{u_i} \sim \mathcal{N}(0,1)$

In [8]:
# generate sample
np.random.seed(1234)
x = np.random.normal(0, 1, 300)
lnu = np.random.normal(0, 1, 300)
y = 3 * np.exp(x + lnu)

Посчитаем оценки $\hat{\beta_1}$ и $\hat{\beta_2}$ (общий вид найден в теоретической части задачи):

$$\hat{\beta_2} = \frac{\sum\limits_{i=1}^{n}x_i(\ln{y_i}-\overline{\ln{y}})}{\sum\limits_{i=1}^{n}x_i(x_i - \overline{x})}$$
$$\ln{\hat{\beta_1}} = \overline{\ln{y}} - \hat{\beta_2}\overline{x} \iff \hat{\beta_1} = \prod\limits_{i=1}^{n}y_i^{\frac{1}{n}} \cdot \exp{(-\hat{\beta_2}\overline{x})}$$

In [9]:
# calculated estimates
n = len(x)
hat_beta2 = np.sum(x * (np.log(y) - np.mean(np.log(y)))) / np.sum(x * (x - np.mean(x)))
hat_beta1 = np.prod(y ** (1 / n)) * np.exp(-hat_beta2 * np.mean(x))

In [10]:
print('Parameters estimates: %.4f, %.4f' %(hat_beta1, hat_beta2))

Parameters estimates: 3.0285, 0.9529


Проверим гипотезу

$$H_0 :
\begin{pmatrix}
    \beta_1 \\
    \beta_2
\end{pmatrix} =
\begin{pmatrix}
    1 \\
    2
\end{pmatrix}
$$
против
$$H_A :
\begin{pmatrix}
    \beta_1 \\
    \beta_2
\end{pmatrix} \neq
\begin{pmatrix}
    1 \\
    2
\end{pmatrix}
$$
на уровне значимости 5% при помощи теста $LR$

In [11]:
# log of likelihood function
def loglike4(beta, x, y):
    n = len(x)
    beta1 = beta[0]
    beta2 = beta[1]
    return - n / 2 * np.log(2 * np.pi) - 1 / 2 * np.sum((np.log(y) - np.log(beta1) - beta2 * x)**2)

In [12]:
# LR test
lr = 2 * (loglike4((hat_beta1, hat_beta2), x, y) - loglike4((1, 2), x, y))
print('Value of LR test: %.4f' % lr)

Value of LR test: 650.5304


Критическое значение $\chi^2_{2}$ равно $5.99 << LR$, значит, гипотеза опровергается.